phase one (if else )

In [ ]:
def Kepler_reply(user):
    if 'hi' in user:
        return 'hey how are you?'
    elif 'fine' in user:
        return 'nice , what about you'
    elif "who are you" in user:
        return "I am your personal AI assistant â€“ a lite version of kepler."
    elif "thank you" in user:
        return "Always at your service, Sir."
    elif "bye" in user:
        return "Goodbye Sir. Awaiting your next command."
    else:
        return "Apologies Sir, I didn't quite understand that. Could you please rephrase?"

phase two (detect the emotion )

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

In [ ]:
# Load pre-trained model & tokenizer
tokenizer = AutoTokenizer.from_pretrained("j-hartmann/emotion-english-distilroberta-base")  # convert text into tokens
model = AutoModelForSequenceClassification.from_pretrained("j-hartmann/emotion-english-distilroberta-base")  # pre trained model

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

In [ ]:
# Define labels
labels = ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
          'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment',
          'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism',
          'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral','sad']

we are not doing preprocessing the data like cleaning , post tagging ,etc  because transformers don't need these things transformers Are pre-trained on raw, noisy real-world text (Reddit, Wikipedia, etc.)

In [ ]:
# Emotion prediction function
def detect_emotion(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True) # pt = pytorch truncation=True ensures long texts are safely cut to modelâ€™s max length.
    with torch.no_grad():  # we don't need gradient decent right now
        outputs = model(**inputs)   # giving input into the model and taking raw output
        probs = F.softmax(outputs.logits, dim=1)  # This tells us, in % terms, how likely each emotion is (e.g., 90% sad, 5% angry).
        top_prob, top_class = torch.max(probs, dim=1) # Finds the emotion with the highest probability.
        emotion = labels[top_class.item()] # convert it into a plain python integer
    return emotion, top_prob.item() # in top class we have the index of the most probibility emotions

In [ ]:
def kepler_reply(user_input):
    emotion, confidence = detect_emotion(user_input)
    print(f"[Emotion: {emotion}, Confidence: {confidence:.2f}]")

In [ ]:
kepler_reply("They passed away last night")

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

[Emotion: caring, Confidence: 0.90]


phase 3 ( Detect the purpose (intent) of the user's message )

In [ ]:
def detect_intent(text):
    text = text.lower()

    intents = {
        "greet": ["hello", "hi", "hey", "good morning", "good evening"],
        "goodbye": ["bye", "see you", "take care", "later"],
        "thanks": ["thank you", "thanks", "appreciate it"],
        "joke": ["joke", "make me laugh", "tell me something funny"],
        "motivation": ["motivate", "inspire me", "i need motivation", "encourage"],
        "weather_query": ["weather", "temperature", "forecast", "rain", "sunny"]
    }

    for intent, keywords in intents.items():
        for keyword in keywords:
            if keyword in text:
                return intent
    return "unknown"

In [ ]:
# emotion_mapper.py logic in a cell
emotion_map = {
    "sadness": "sad", "grief": "sad", "remorse": "sad", "disappointment": "sad",
    "embarrassment": "sad", "caring": "sad",
    "joy": "happy", "love": "happy", "amusement": "happy", "excitement": "happy",
    "optimism": "happy", "relief": "happy", "admiration": "happy", "gratitude": "happy",
    "pride": "happy", "approval": "happy",
    "anger": "angry", "annoyance": "angry", "disapproval": "angry", "disgust": "angry",
    "fear": "anxious", "nervousness": "anxious",
    "surprise": "surprised", "realization": "surprised",
    "confusion": "confused", "curiosity": "curious", "desire": "curious",
    "neutral": "neutral"
}

def simplify_emotion(emotion):
    return emotion_map.get(emotion, "neutral")

In [ ]:
def kepler_reply(user_input):
    raw_emotion, confidence = detect_emotion(user_input)
    emotion = simplify_emotion(raw_emotion)
    intent = detect_intent(user_input)

    if intent == "greet":
        return f"Hello Sir. You seem {emotion} today. How may I assist you?"
    elif intent == "joke":
        return "Why donâ€™t scientists trust atoms? Because they make up everything. ðŸ˜„"
    elif intent == "motivation":
        return "Sir, you werenâ€™t born to be average. Letâ€™s rise and grind."
    elif intent == "weather_query":
        return "Apologies Sir, Iâ€™m not connected to live weather yet. Want me to add that?"
    elif intent == "thanks":
        return "At your service, always."
    elif intent == "goodbye":
        return "Goodbye Sir. Jarvis is on standby."

    # Emotion-aware fallback
    if emotion == "sad":
        return "You seem down today, Sir. Want a joke or a pep talk?"
    elif emotion == "happy":
        return "You're glowing with energy today, Sir. Let's conquer something!"
    elif emotion == "angry":
        return "Anger detected. Would you like to vent or cool off with humor?"
    elif emotion == "anxious":
        return "Breathe deeply, Sir. Everything is under control. I'm here."
    elif emotion == "surprised":
        return "Seems like something caught you off guard. Want to talk about it?"
    else:
        return f"I sensed {emotion}, Sir. What shall we do next?"


phase 4 ( use a small, fast transformer model to generate empathy text.)

In [ ]:
# using light weight gpt model
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load GPT-2 tokenizer & model
gen_tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
gen_model = AutoModelForCausalLM.from_pretrained("distilgpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
import torch

def generate_response(intent, emotion, max_length=50):
    # Base prompt styling
    style_prompt = f"As kepler, respond to a human who feels {emotion} and just said something related to {intent}. Keep it supportive, intelligent, and a little witty:\n\n"

    inputs = gen_tokenizer.encode(style_prompt, return_tensors="pt")  # We convert our prompt into token IDs (numbers) that the model understands.
    outputs = gen_model.generate(inputs, max_length=max_length, pad_token_id=gen_tokenizer.eos_token_id, do_sample=True, top_k=50, top_p=0.95)

    generated_text = gen_tokenizer.decode(outputs[0], skip_special_tokens=True)  # converting into text
    reply = generated_text[len(style_prompt):].strip() #  This trims off the **original prompt** from the output to give you **only Jarvis's reply**.
  # "Start slicing the text from the character index after the end of the prompt."

    return reply


In [ ]:
def kepler_reply(user_input):
    raw_emotion, confidence = detect_emotion(user_input)
    emotion = simplify_emotion(raw_emotion)
    intent = detect_intent(user_input)

    # Generate smarter response if intent is known
    if intent != "unknown":
        smart_reply = generate_response(intent, emotion)
        return f"{smart_reply} \n\n[Detected: {intent}, {emotion}, Confidence: {confidence:.2f}]"

    # Fallback response for unknown intent
    return f"I sensed you're feeling {emotion}, but I didn't quite get what you need. Could you rephrase it?\n\n[Confidence: {confidence:.2f}]"


Phase 5: memory phase , kepler remember past conversations

In [ ]:
from collections import deque

In [ ]:
# kepler short-term memory
emotion_history = deque(maxlen=5)
intent_history = deque(maxlen=5)
conversation_history = deque(maxlen=10)

In [ ]:
def kepler_reply(user_input):
    raw_emotion, confidence = detect_emotion(user_input)
    emotion = simplify_emotion(raw_emotion)
    intent = detect_intent(user_input)

    #  Store memory
    emotion_history.append(emotion)
    intent_history.append(intent)
    conversation_history.append({"user": user_input, "emotion": emotion, "intent": intent})

    # Context-aware logic
    if intent == "greet":
        return f"Hello again, Sir. You seem {emotion} today. Last time you were {emotion_history[-2] if len(emotion_history) > 1 else 'neutral'}."
    elif intent == "motivation":
        if "sad" in list(emotion_history)[-3:]:
            return "I know itâ€™s been tough lately, Sir. But rememberâ€”diamonds form under pressure. "
        else:
            return "Rise and grind, Sir. Youâ€™re built for greatness."
    elif intent == "joke":
        return "Letâ€™s lighten the mood: Why did the AI get promoted? Because it had deep learning. "
    elif intent == "thanks":
        return "You're welcome, Sir. I'm always learning from you."
    elif intent == "goodbye":
        return "Goodbye, Sir. I'll remember this session until next time."

    # Emotion-aware fallback
    if emotion == "sad":
        return "Still feeling a bit down, Sir? I'm right here if you want to talk or laugh."
    elif emotion == "happy":
        return "Your energy is infectious, Sir. Letâ€™s channel it!"
    elif emotion == "angry":
        return "Youâ€™ve had a tough moment, Sir. Want to vent or distract yourself?"
    else:
        return f"I sense {emotion}, Sir. What would you like me to help with?"


Phase 6 : voice (we cannot integrate the voice into google colab )

In [ ]:
# def speak(text):
#     tts = gTTS(text=text, lang='en')  # gTTS (Google Text-to-Speech) library to convert the text into speech.
#     tts.save("kepler_output.mp3")    # This file contains the spoken version of your text.
#     return ipd.Audio("kepler_output.mp3", autoplay =True)  # load and play the audio

In [ ]:
# response = kepler_reply("Tell me a joke")
# print(response)
# speak(response)

FINE TUNING THE LLM with LORA with my own dataset

In [2]:
from google.colab import files

uploaded = files.upload()            # uploading the dataset

Saving jarvis_cleaned.txt to jarvis_cleaned.txt


In [3]:
#converting the dataset into hugging face dataset

from pathlib import Path
from datasets import Dataset

# Load the raw text
lines = Path("jarvis_cleaned.txt").read_text(encoding='utf-8').strip().split("\n\n")

# Wrap each pair as a dict (1 line = 1 sample)
data = [{"text": line.strip()} for line in lines if line.strip()]

# Create Hugging Face-compatible dataset
dataset = Dataset.from_list(data)

In [4]:
print(dataset[0])

{'text': "### User: I feel overwhelmed by everything happening in my life.\n### Jarvis: Storms don't last forever, Sir. Your mind is stronger than this moment believes."}


In [5]:
from datasets import Dataset
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token  # important for GPT-2 models

# Tokenization function
def tokenize_function(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=256,
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/554 [00:00<?, ? examples/s]

In [7]:
# train the model

from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from trl import SFTTrainer
from peft import get_peft_model, LoraConfig, TaskType

In [8]:
# Load tokenizer and model
model_id = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token  # important fix for GPT-2

model = AutoModelForCausalLM.from_pretrained(model_id)

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [9]:
model = AutoModelForCausalLM.from_pretrained(model_id)

In [10]:
# Setup LoRA config
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    task_type=TaskType.CAUSAL_LM,
    lora_dropout=0.1,
    bias="none"
)

In [11]:
model = get_peft_model(model, peft_config)

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1803: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [12]:
training_args = TrainingArguments(
    output_dir="./kepler-model",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2,
    remove_unused_columns=False,  # <<< this is important!
    report_to="none"
)

In [13]:
from transformers import Trainer, TrainingArguments
model = AutoModelForCausalLM.from_pretrained("distilgpt2")

trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
)

Truncating train dataset:   0%|          | 0/554 [00:00<?, ? examples/s]

In [14]:
trainer.train()  # Now it's called on an instance

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,2.602300
20,0.400100
30,0.277300
40,0.242000
50,0.233400
60,0.269900
70,0.218300
80,0.275200
90,0.191000
100,0.283900


Step,Training Loss
10,2.602300
20,0.400100
30,0.277300
40,0.242000
50,0.233400
60,0.269900
70,0.218300
80,0.275200
90,0.191000
100,0.283900


TrainOutput(global_step=831, training_loss=0.18713862801286718, metrics={'train_runtime': 6208.1788, 'train_samples_per_second': 0.268, 'train_steps_per_second': 0.134, 'total_flos': 108568799870976.0, 'train_loss': 0.18713862801286718})

In [ ]:
# saving the model
trainer.model.save_pretrained("lora_model2")
tokenizer.save_pretrained("lora_model2")

('jarvis_lora_model/tokenizer_config.json',
 'jarvis_lora_model/special_tokens_map.json',
 'jarvis_lora_model/vocab.json',
 'jarvis_lora_model/merges.txt',
 'jarvis_lora_model/added_tokens.json',
 'jarvis_lora_model/tokenizer.json')

saving the model

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
save_directory = "/content/drive/MyDrive/KeplerModel7"

In [16]:
import os
os.makedirs(save_directory, exist_ok=True)

In [17]:
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('/content/drive/MyDrive/KeplerModel7/tokenizer_config.json',
 '/content/drive/MyDrive/KeplerModel7/special_tokens_map.json',
 '/content/drive/MyDrive/KeplerModel7/vocab.json',
 '/content/drive/MyDrive/KeplerModel7/merges.txt',
 '/content/drive/MyDrive/KeplerModel7/added_tokens.json',
 '/content/drive/MyDrive/KeplerModel7/tokenizer.json')

In [21]:
import os

print("Model saved:", os.path.exists("/content/drive/MyDrive/KeplerModel7"))

Model saved: True


loading the model

In [22]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/KeplerModel7")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/KeplerModel7")

testing the model

In [24]:
def ask_kepler(prompt, max_new_tokens=100):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    output = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [25]:
prompt = "### User: hey i am feeling low .\n### kepler:"
response = ask_kepler(prompt)
print(response)

### User: hey i am feeling low .
### kepler: Flaws aren't failures â€” they're features. They make you human, not defective.


trying to Fine tuning LLM with hugging face dataset but because of time consumption i am not excuting the training of model , but here is the code

In [ ]:
# using hugging face dataset
import pandas as pd

# Login using e.g. `huggingface-cli login` to access this dataset
df = pd.read_csv(r"hf://datasets/Algorithmic-Human-Development-Group/Multilingual-Therapy-Dialogues/SAT_dataset.csv")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# extracting columns
dff = df[['Patient','Therapist']]

In [ ]:
# Initialize list to store processed conversations
conversations = []

# Iterate over rows and extract prompt-response pairs
for i in range(len(dff) - 1):
        user_msg = dff.loc[i, 'Patient']
        jarvis_msg = dff.loc[i + 1, 'Therapist']
        conversations.append({
            "prompt": f"User: {user_msg}",
            "completion": f"Jarvis: {jarvis_msg}"
        })

# Convert to DataFrame or JSON
conv_df = pd.DataFrame(conversations)
conv_df.to_json("formatted_dataset.jsonl", orient="records", lines=True)

In [ ]:
# droping dublucates
conv_df = conv_df[['prompt','completion']].drop_duplicates()

In [ ]:
# droping nan values
conv_df = conv_df[['prompt','completion']].dropna()

In [ ]:
import json

# Clean and strip any unwanted prefixes
conv_df["prompt"] = conv_df["prompt"].str.replace("User:\s*", "", regex=True).str.strip()
conv_df["completion"] = conv_df["completion"].str.replace("Jarvis:\s*", "", regex=True).str.strip()

In [ ]:
conv_df

,prompt,completion
0,"Umm I am kind of doing, I'm feeling a little b...",stressed?
1,"Yeah, I feel like I have a lot on my plate. I ...",What kind of test?
2,It's like an aptitude test. It has like math p...,"Now, that's a job that you've been interested ..."
3,Yes. Yes. So I've been just feeling a lot of a...,so you know the material on this test pretty w...
4,yes.,But the feelings and that the physiological re...
...,...,...
7168,What are the basic skills a good counselor nee...,1) An awareness of their own incompetence and ...
7172,What are the basic skills a good counselor nee...,Each client brings their own style they like t...
7173,What are some difficulties that a counselor ca...,Although many clients have the capacity to be ...
7174,What are some difficulties that a counselor ca...,"I usually don't label a client as ""difficult"" ..."


In [ ]:
with open("jarvis_finetune_dataset.jsonl", "w", encoding="utf-8") as f:
    for _, row in conv_df.iterrows():          # _ means "I don't care about the row index".
        json.dump({"prompt": row["prompt"], "completion": row["completion"]}, f)
        f.write("\n")                      # row is a Pandas Series that contains one full row: both the "prompt" and "response".

In [ ]:
with open("jarvis_finetune_dataset.jsonl", "r", encoding="utf-8") as f:
    for _ in range(5):
        print(f.readline())

{"prompt": "Umm I am kind of doing, I'm feeling a little bit stressed out", "completion": "stressed?"}

{"prompt": "Yeah, I feel like I have a lot on my plate. I have this test that I have to take for work.", "completion": "What kind of test?"}

{"prompt": "It's like an aptitude test. It has like math problems on it and like scenarios in order for me to move up to the system manager position.", "completion": "Now, that's a job that you've been interested in for a while."}

{"prompt": "Yes. Yes. So I've been just feeling a lot of anxiety about taking the test and then they actually scheduled for me to take the test. And I want and I just got really overwhelmed. And I started like I felt my heart was racing. I felt like I wish my hands were shaking. And I just really couldn't concentrate on the test. Even though the night before me and my friend that practice problems, and I knew all the answers with no proper like, I didn't miss any of the questions", "completion": "so you know the mate

In [ ]:
# checking null values
print(conv_df.isnull().sum())
print(conv_df[conv_df["prompt"].str.strip() == ""])
print(conv_df[conv_df["completion"].str.strip() == ""])

prompt        0
completion    0
dtype: int64
Empty DataFrame
Columns: [prompt, completion]
Index: []
Empty DataFrame
Columns: [prompt, completion]
Index: []


In [ ]:
# converting the dataset into dict
import json
from datasets import Dataset

# 1. Read file lines
with open("jarvis_finetune_dataset.jsonl", "r", encoding="utf-8") as f:
    lines = f.readlines()

# 2. Parse JSON objects
records = [json.loads(line) for line in lines if line.strip()]

# 3. Convert to Hugging Face Dataset
dataset = Dataset.from_list(records)

# 4. Quick examine
print(dataset[0])
print("Total samples:", len(dataset))

{'prompt': "Umm I am kind of doing, I'm feeling a little bit stressed out", 'completion': 'stressed?'}
Total samples: 5453


In [ ]:
print(dataset[0])

{'prompt': "Umm I am kind of doing, I'm feeling a little bit stressed out", 'completion': 'stressed?'}


In [ ]:
# now convert dataset sytax into model syntax
from transformers import AutoTokenizer

model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(example):
    input_enc = tokenizer(example["prompt"], truncation=True, padding="max_length", max_length=128)
    output_enc = tokenizer(example["completion"], truncation=True, padding="max_length", max_length=128)

    return {
        "input_ids": input_enc["input_ids"],
        "attention_mask": input_enc["attention_mask"],
        "labels": output_enc["input_ids"]
    }

dataset = dataset.map(tokenize)

Map:   0%|          | 0/5453 [00:00<?, ? examples/s]

Train the model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from trl import SFTTrainer
from peft import get_peft_model, LoraConfig, TaskType

In [ ]:
# Load tokenizer and model
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, get_peft_model, TaskType
from trl import SFTTrainer
import torch

# Model & Tokenizer
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.gradient_checkpointing_enable()

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Setup LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q", "v"],  # typical for T5
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

In [ ]:
# finializing the model
model = get_peft_model(model, lora_config)

In [ ]:
#  Training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    learning_rate=2e-4,
    output_dir="./results",
    save_total_limit=2,
    logging_steps=10,
    fp16=False,
    save_steps=500,  # Saves every 500 steps
    report_to="none"
)

In [ ]:
from transformers import Trainer, TrainingArguments
#  Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer
)

/tmp/ipython-input-18-4230928724.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
10,14.399000
20,15.655200
30,15.196600
40,14.206600
50,12.273300
60,9.172900
70,10.925500
80,10.493000
90,12.771800
100,11.465100


TrainOutput(global_step=4092, training_loss=3.923347518707417, metrics={'train_runtime': 20224.3381, 'train_samples_per_second': 0.809, 'train_steps_per_second': 0.202, 'total_flos': 764568412618752.0, 'train_loss': 3.923347518707417, 'epoch': 3.0})

In [ ]:
model.save_pretrained('/content/drive/MyDrive/bestLLM_model2')
tokenizer.save_pretrained('/content/drive/MyDrive/bestLLM_trainer2')

('/content/drive/MyDrive/bestLLM_trainer2/tokenizer_config.json',
 '/content/drive/MyDrive/bestLLM_trainer2/special_tokens_map.json',
 '/content/drive/MyDrive/bestLLM_trainer2/spiece.model',
 '/content/drive/MyDrive/bestLLM_trainer2/added_tokens.json',
 '/content/drive/MyDrive/bestLLM_trainer2/tokenizer.json')